In [1]:
import torch
#!pip install facenet_pytorch
from facenet_pytorch import MTCNN
 
import cv2
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt
 
import pandas as pd
 
import albumentations as A
from albumentations.pytorch import ToTensorV2 
import torch.nn as nn
from torch.utils.data import Dataset 
from torchvision.models import resnet50 
import torch.nn.functional as F
import torchvision
from torchvision.datasets import ImageFolder
from torch.utils.data import   DataLoader, random_split
import torchvision.transforms as transforms
import os

ModuleNotFoundError: No module named 'facenet_pytorch'

In [12]:
class FaceDetector:
    def __init__(self, weights_path, device=None):
        if device is None:
            self.device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
        else:
            self.device = device
        self.model = MTCNN(keep_all=True, device=self.device)
        self.load_weights(weights_path)

    def load_weights(self, weights_path):
        # Загрузка весов в модель
        state_dict = torch.load(weights_path, map_location=self.device)
        self.model.load_state_dict(state_dict)

    def detect_faces(self, image):        
        boxes, _ = self.model.detect(image)
        boxes = boxes.astype(int)

        # Делаю padding
        boxes[:, 0] -= 10
        boxes[:, 1] -= 10
        boxes[:, 2] += 20
        boxes[:, 3] += 20

        return boxes

    def box_faces(self, image):
        boxes = self.detect_faces(image)
       
        img = Image.fromarray(img.astype('uint8'), 'RGB')
        result = []

        for box in boxes:
            x_left, y_left = box[0], box[1]
            x_right, y_right = box[2], box[3]

            face_area = img.crop((x_left, y_left, x_right, y_right))
            face_area = np.array(face_area, dtype='uint8')
            result.append(face_area)

        return result

    def display_faces(self, image):
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        boxes = self.detect_faces(image)
        image_np = np.array(image)

        fig, ax = plt.subplots()
        ax.imshow(image_np)

        if boxes is not None:
            for box in boxes:
                rect = plt.Rectangle((box[0], box[1]), box[2] - box[0], box[3] - box[1], fill=False, color='red')
                ax.add_patch(rect)

        plt.axis('off')
        plt.show()

In [13]:
class SimpleCNNModel(nn.Module) :
    def __init__(self):
        super().__init__()
        self.model = resnet50(pretrained=True)
        self.model.fc = nn.Linear(2048, 68*2)

    def forward(self, x) :
        output = self.model(x)
        return output

NameError: name 'nn' is not defined

In [ ]:
class Normalizer():
    def __init__(self, model, img):
        self.model = model
        self.transform = A.Compose([
            A.Resize(220, 220),
            ToTensorV2()
        ])

        self.img = self.transform(image=img)['image']


    def coord_landmarks(self):
        predict = self.model(torch.tensor(np.expand_dims(self.img, axis=0)).float())

        num_landmarks = [i for i in range(16, -1, -1)] + [i for i in range(17, 20)] + [i for i in range(24, 27)] + [16]

        x_pred = predict.detach().numpy()[0, ::2][num_landmarks]
        y_pred = predict.detach().numpy()[0, 1::2][num_landmarks]

        x_pred = x_pred.reshape(-1, 1)
        y_pred = y_pred.reshape(-1, 1)

        coords = np.concatenate((x_pred, y_pred), axis=1)

        return coords


    def show_landmarks(self):
        coords = self.coord_landmarks()
        x = coords[:, 0]
        y = coords[:, 1]
        size_points = 10

        img = self.img.numpy()[0]

        plt.scatter(x, y, s=size_points)
        plt.imshow(img, cmap='gray')


    def normalizer(self):
        coords = self.coord_landmarks().astype(int)
        self.img = self.img.numpy()[0]

        mask = np.zeros((self.img.shape[0], self.img.shape[1]))
        mask = cv2.fillConvexPoly(mask, coords, (255, 255, 255))
        mask = mask.astype(bool)

        out = np.zeros_like(self.img)
        out[mask] = self.img[mask]

        plt.imshow(out, cmap='gray')
        plt.axis('off')

        return out


In [2]:
class Extractor:
    def __init__(self, model):
        self.model = model

    def transform_image(self, image):
        transform = transforms.Compose([transforms.Resize((128, 128)),transforms.ToTensor(),transforms.Grayscale(),])
        
        image = transform(image)
     
        image = image.unsqueeze(0)
        return image
         
         
    def get_features(self, image):
        image = self.transform_image(image)    
        
        features = model(image)         
        return features

In [ ]:
class Check_similarity():
    def __init__(self, image_path1, image_path2):
        self.image1 = cv2.imread(image_path1)
        self.image2 = cv2.imread(image_path2)         
        
    
    def detect(self):
        detector = FaceDetector()
        face1 = detector.box_faces(self.image1)
        face2 = detector.box_faces(self.image2)
        return face1, face2
    
    def normalization(self):
        face1, face2 = self.detect() 
        model = torch.load('\model_normalizer')
        normalizer1 = Normalizer(model, face1)
        normalizer2 = Normalizer(model, face2)
        norm1 = normalizer1.noramlizer()
        norm2 = normalizer2.noramlizer()
        return norm1, norm2
    
    def extract(self):
        face1, face2 = self.normalization()
        model = torch.load('\model1.pth')
        
        extractor = Extractor(model)
        features1 = extractor.get_features(face1)
        features2 = extractor.get_features(face2)
        
        return features1, features2
    
    
    def similarity_score(self):    
        features1, features2 = self.extract()
        euclidean_dist = torch.norm(features1 - features2, p=2) 
        similarity = torch.exp(-euclidean_dist)
    
        return similarity 